In [9]:
import socket
import threading

socket.gethostbyname_ex(socket.gethostname())

('DESKTOP-AG6MLBM', [], ['169.254.49.37', '192.168.56.1', '192.168.0.54'])

In [ ]:
import socket
import threading

HEADER = 64
PORT = 5050
SERVER = socket.gethostbyname_ex(socket.gethostname())[2][2]
ADDR = (SERVER, PORT)
FORMAT = 'utf-8'
DISCONNECT_MESSAGE = "!DISCONNECT"

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(ADDR)



def handle_client(conn: socket.socket, addr):
    print(f"[NEW CONNECTION] {addr} connected")
    
    connected = True
    while connected:
        msg_lenght = conn.recv(HEADER).decode(FORMAT)
        if msg_lenght:            
            msg_lenght = int(msg_lenght)
            msg = conn.recv(msg_lenght).decode(FORMAT) 
            if msg == DISCONNECT_MESSAGE:
                connected = False
            print(f"{addr} {msg_lenght} {msg}")
            conn.send("Msg received".encode(FORMAT))

def start():
    server.listen()
    print(f"[LISTENING] Server is listening in {SERVER}")
    while True:
        conn, addr = server.accept()
        thread = threading.Thread(target=handle_client, args=(conn, addr))
        thread.start()
        print(f"[ACTIVE CONNECTIONS] {threading.activeCount() - 1}")
        

: 

In [2]:
print("[STARTING] server is starting...")
start()

[STARTING] server is starting...
[LISTENING] Server is listening in 192.168.0.54
[NEW CONNECTION] ('192.168.0.54', 4365) connected[ACTIVE CONNECTIONS] 6

('192.168.0.54', 4365) 11 HOLA MUNDO!
('192.168.0.54', 4365) 11 hola chicho
('192.168.0.54', 4365) 11 !DISCONNECT
[NEW CONNECTION] ('192.168.0.54', 4416) connected[ACTIVE CONNECTIONS] 6

('192.168.0.54', 4416) 11 HOLA MUNDO!
('192.168.0.54', 4416) 11 hola chicho
('192.168.0.54', 4416) 11 !DISCONNECT


In [5]:
import socket

PORT = 32000
socket.gethostbyname_ex(socket.gethostname())[-1][-1]

'192.168.0.54'

In [128]:
incremental = 0

print(f'{incremental:04d}')

0000


In [342]:
import socket

PORT = 2001
SERVER = socket.gethostbyname_ex(socket.gethostname())[-1][-1]
ADDR = (SERVER, PORT)

server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server.bind(ADDR)

incremental+=1

print(f'DE,{incremental:04d},0000,CID,0056181122010030')

msg_to_send = encrypt_bap_msg(f'DE,{incremental:04d},0000,CID,0056181122010030')

server.sendto(bytes(msg_to_send), ('192.168.0.10', 2000))

datos = server.recvfrom(2048)

print(datos)

server.close()


DE,0214,0000,CID,0056181122010030
(b'\x01$e\xcc;CWm\x82\x92', ('192.168.0.10', 2000))


In [110]:
import random

def encrypt_bap_msg(msg: str) -> str: 
    aux = []
    result = []
    
    shkey = random.randint(0,255)        
    
    for i in msg:
        aux.append(ord(i) ^ shkey) 
    
    for i in range(1,len(aux)):
        aux[i] = aux[i] + aux[i-1] & 0xFF        
        
    result.append(1)
    result.append(shkey)
    result.extend(aux)
    
    return result
    
encrypt_bap_msg('hola como estas')

[1, 184, 208, 167, 123, 84, 236, 199, 158, 115, 74, 226, 191, 138, 86, 47, 250]

In [344]:
server, client = socket.socketpair(socket.AF_INET, socket.SOCK_DGRAM)

ValueError: Only SOCK_STREAM socket type is supported

In [343]:
import socket
import asyncio
import os, random

HOST, PORT = 'localhost', 514


def send_test_message(message: 'Message to send to UDP port 514') -> None:
    sock = socket.socket(socket.AF_INET,  # Internet
                         socket.SOCK_DGRAM)  # UDP
    sock.sendto(message.encode(), (HOST, PORT))


async def write_messages() -> "Continuously write messages to UDP port 514":
    dir_path = os.path.dirname(os.path.realpath(__file__))
    fp = open(os.path.join(dir_path, "tests/example.log"))
    print("writing")
    for line in fp.readlines():
        await asyncio.sleep(random.uniform(0.1, 3.0))
        send_test_message(line)


class SyslogProtocol(asyncio.DatagramProtocol):
    def __init__(self):
        super().__init__()

    def connection_made(self, transport) -> "Used by asyncio":
        self.transport = transport

    def datagram_received(self, data, addr) -> "Main entrypoint for processing message":
        # Here is where you would push message to whatever methods/classes you want.
        print(f"Received Syslog message: {data}")


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    t = loop.create_datagram_endpoint(SyslogProtocol, local_addr=('0.0.0.0', PORT))
    loop.run_until_complete(t) # Server starts listening
    loop.run_until_complete(write_messages()) # Start writing messages (or running tests)
    loop.run_forever()

RuntimeError: This event loop is already running

In [375]:
from numpy import uint8

mensajes = [
b'\x01\x17V\xaa\x06Ap\x92\xb3\xe1',
b'\x01u4j\xa8\x01N\x8e\xd0\x15',
b'\x01\xa7\xe6\xca\xb6A\xe0r\x02\x98',
b'\x01\xc9\x88\x12\x94yjfd_',
b'\x01c"Bj\xb9\x14j\xbe\x0e',
b'\x01h)Tw\xbb\x0bh\xc7#',
b'\x01\x01@\x82\xcc\xf92f\x9c\xd0',
b'\x01)h\xd249Jf\x84\xa3',
b'\x01\x93\xd2\xa2z9\xe4\x8a.\xd2',
b'\x01K\n\x12\x12y\xecj\xe6Y',
b'\x011p\xe2\\y\x82\x86\x8c\x94',
b'\x01\x12S\xa4\xfd;e\x8c\xb6\xd8'
]

def decrypt_bap_msg(MsgBapMgr) -> str:
    result = []
    if len(MsgBapMgr) > 3:
        bkey = MsgBapMgr[1]										# El segundo byte es un caracter random generado por el que envia el mensaje
        MsgBapMgr[1] = 0										# Cuando se encripta el mensaje, al primer elemento se le aplica XOR pero no se suma con nada!!!!*

        for i in range(2,len(MsgBapMgr)):
            result.append((uint8(MsgBapMgr[i]) - uint8(MsgBapMgr[i-1])) ^ bkey)                 
        
        return bytes(result).decode()
		
for m in mensajes:
    decr = decrypt_bap_msg(bytearray(m))
    print(decr)

ACK,8569
ACK,8570
ACK,8571
ACK,8572
ACK,8573
ACK,8574
ACK,8575
ACK,8576
ACK,8577
ACK,8578
ACK,8579
ACK,8580


C:\Users\fert2\AppData\Local\Temp\ipykernel_27704\372282375.py:25: RuntimeWarning: overflow encountered in ubyte_scalars
  result.append((uint8(MsgBapMgr[i]) - uint8(MsgBapMgr[i-1])) ^ bkey)


In [174]:
import random

def calcCidChecksum(cidmsg):
    suma=0
    for i in cidmsg:
        suma += (int(i,base=16) if (i != '0') else 10)
        resto = 15 - (suma % 15)
        #print(f'{i}  {suma}  {hex(suma)}  {hex(resto)}')
    return resto
        
def rndCidCreator():
    acct = random.choice([4,7])#random.randint(1,9999)
    q = random.choice([1,3])
    evc = random.randint(1,999)
    gg = random.randint(1,32)
    CCC = random.randint(1,999)
    cidmsg = f'{acct:04d}18{q}{evc:03d}{gg:02d}{CCC:03d}'
    chksum = calcCidChecksum(cidmsg)
    print(f'{cidmsg}{chksum:X}')

rndCidCreator()
    

000718123808528C
